In [1]:
import geopandas as gpd
from shapely.geometry import Polygon
import pyproj
import math
import os
import zipfile
import numpy as np

In [2]:
def create_bounding_square_shapefile(input_shapefile, buffer_m=10):
    """
    Create a square shapefile that fully covers all points in an input shapefile,
    with an additional buffer.

    Parameters:
    - input_shapefile: Path to the input point shapefile.
    - buffer_m: Buffer in meters (default is 10m).

    Returns:
    - None
    """
    # Load the point shapefile
    gdf = gpd.read_file(input_shapefile)
    
    # Get the CRS from the input shapefile
    input_crs = gdf.crs
    if input_crs is None:
        raise ValueError("The input shapefile has no defined CRS.")
    
    # Ensure it contains only point geometries
    if not all(gdf.geometry.type == "Point"):
        raise ValueError("The input shapefile must contain only Point geometries.")
    
    # Compute centroid of all points
    centroid = gdf.geometry.union_all().centroid  # Updated for GeoPandas 0.14+
    center_x, center_y = centroid.x, centroid.y
    
    # Transform all points to projected CRS (if not already projected)
    if input_crs.is_geographic:
        utm_zone = (math.floor((center_x + 180) / 6) % 60) + 1
        if center_y >= 0:
            utm_crs = pyproj.CRS(f'EPSG:326{utm_zone:02d}')  # Northern hemisphere
        else:
            utm_crs = pyproj.CRS(f'EPSG:327{utm_zone:02d}')  # Southern hemisphere
    else:
        utm_crs = input_crs
    
    project_to_utm = pyproj.Transformer.from_crs(input_crs, utm_crs, always_xy=True).transform
    project_to_input = pyproj.Transformer.from_crs(utm_crs, input_crs, always_xy=True).transform
    
    center_utm = project_to_utm(center_x, center_y)
    
    # Transform all points to UTM
    gdf_utm = gdf.to_crs(utm_crs)
    
    # Compute max distance from centroid to any point
    #centroid_utm = gdf_utm.geometry.unary_union.centroid
    centroid_utm = gdf_utm.geometry.union_all().centroid

    distances = gdf_utm.geometry.distance(centroid_utm)
    if distances.isna().any():
        raise ValueError("Distance calculation resulted in NaN values. Check input data.")
    max_distance = max(distances)
    
    # Compute square side length
    side_length = 2 * (max_distance + buffer_m)  # Ensure square covers all points + buffer
    
    # Compute the four corners of the square in UTM coordinates
    half_side = side_length / 2
    square_utm_coords = [
        (center_utm[0] - half_side, center_utm[1] - half_side),
        (center_utm[0] + half_side, center_utm[1] - half_side),
        (center_utm[0] + half_side, center_utm[1] + half_side),
        (center_utm[0] - half_side, center_utm[1] + half_side),
        (center_utm[0] - half_side, center_utm[1] - half_side)  # Ensure polygon closure
    ]
    
    # Validate UTM coordinates
    if any(np.isnan(coord[0]) or np.isnan(coord[1]) for coord in square_utm_coords):
        raise ValueError("Generated square contains NaN values. Check UTM transformations.")
    
    square_utm = Polygon(square_utm_coords)
    
    # Convert the square back to the original CRS
    square_input_coords = [project_to_input(x, y) for x, y in square_utm.exterior.coords]
    square_input = Polygon(square_input_coords)
    
    # Validate final polygon
    if not square_input.is_valid:
        raise ValueError("Generated square is invalid after re-projection. Check calculations.")
    
    # Create GeoDataFrame
    gdf_square = gpd.GeoDataFrame(index=[0], crs=input_crs, geometry=[square_input])
    
    
    # Extract the directory containing the file
    input_filename = os.path.splitext(os.path.basename(input_shapefile))[0]
    print(f'input_filename: {input_filename}')

    output_folder_path = os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(input_shapefile))),"cut_shapefile", f"{input_filename}")
    print(f'output_folder_path: {output_folder_path}')


    # Create output directory
    os.makedirs(output_folder_path, exist_ok=True)
    
    # Define output file paths
    shapefile_path = os.path.join(output_folder_path, f"{input_filename}_cutshape.shp")
    zipfile_path = os.path.join(output_folder_path, f"{input_filename}_cutshape.zip")
    
    # Save to shapefile
    gdf_square.to_file(shapefile_path)
    print(f"Bounding square shapefile saved to {shapefile_path}")
    
    # Create a zip archive of the folder
    with zipfile.ZipFile(zipfile_path, 'w') as zipf:
        for root, _, files in os.walk(output_folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, output_folder_path)
                zipf.write(file_path, arcname=arcname)
    
    print(f"Zipfile created at {zipfile_path}")

In [3]:
# Example usage:
input_shapefile = "D:/MasterThesis/filnal_hs_data_folder/ang20180812t221620rfl/data/plotlocation_shapefile/FRST_AK_Plot1/FRST_AK_Plot1.shp"  # Replace with your actual shapefile
# Check if files exist
if os.path.exists(input_shapefile):
    print(f"File exists: {input_shapefile}")
else:
    print(f"File not found: {input_shapefile}")

File exists: D:/MasterThesis/filnal_hs_data_folder/ang20180812t221620rfl/data/plotlocation_shapefile/FRST_AK_Plot1/FRST_AK_Plot1.shp


In [4]:
create_bounding_square_shapefile(input_shapefile, buffer_m=10)

input_filename: FRST_AK_Plot1
output_folder_path: D:/MasterThesis/filnal_hs_data_folder/ang20180812t221620rfl/data\cut_shapefile\FRST_AK_Plot1
Bounding square shapefile saved to D:/MasterThesis/filnal_hs_data_folder/ang20180812t221620rfl/data\cut_shapefile\FRST_AK_Plot1\FRST_AK_Plot1_cutshape.shp
Zipfile created at D:/MasterThesis/filnal_hs_data_folder/ang20180812t221620rfl/data\cut_shapefile\FRST_AK_Plot1\FRST_AK_Plot1_cutshape.zip


In [5]:
input_shapefile = "D:/MasterThesis/filnal_hs_data_folder/ang20180812t221620rfl/data/plotlocation_shapefile/FRST_AK_Plot1/FRST_AK_Plot1.shp"
parent_dir = os.path.dirname(os.path.dirname(os.path.dirname(input_shapefile)))
print(parent_dir)

D:/MasterThesis/filnal_hs_data_folder/ang20180812t221620rfl/data


In [6]:
import os

filepath = "D:/MasterThesis/filnal_hs_data_folder/ang20180812t221620rfl/data/plotlocation_shapefile/FRST_AK_Plot1/FRST_AK_Plot1.shp"

# Extract the directory containing the file
dirname = os.path.dirname(filepath)

# Extract the filename without the extension
filename = os.path.splitext(os.path.basename(filepath))

# Print the filename
print(filename[0])  # Output: FRST_AK_Plot1

FRST_AK_Plot1
